In [40]:
import pandas as pd
from pandas import DataFrame
import dateutil.parser

view_party=pd.read_csv('../refdata/view_party.csv').drop(['cmp','euprofiler','ees','castles_mair','huber_inglehart','ray','benoit_laver','chess'],axis=1)
partyColumns=view_party.columns

view_election=pd.read_csv('../refdata/view_election.csv')
electionColumns=view_election.columns

view_election=view_election.merge(view_party,on='party_id',how='left',suffixes=('','_2'))
view_cabinet=pd.read_csv('../refdata/view_cabinet.csv')

view_election=view_election.drop([x for x in view_election.columns if x.endswith('_2')],axis=1)
view_election.election_date=view_election.election_date.apply(dateutil.parser.parse)

NationalElections=view_election[view_election.election_type=='parliament']
EuroElections=view_election[view_election.election_type=='ep']


def getLastElection(t):
    return t[t.election_date==t.election_date.max()]

NationalElections=NationalElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)
EuroElections=EuroElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)

NationalElections.to_csv('../refdata/national_elections.csv',index=False)
EuroElections.to_csv('../refdata/euro_elections.csv',index=False)

In [45]:
electionColumns


Index([u'country_name_short', u'country_name', u'election_type',
       u'election_date', u'vote_share', u'seats', u'seats_total',
       u'party_name_short', u'party_name', u'party_name_english',
       u'left_right', u'country_id', u'election_id',
       u'previous_parliament_election_id', u'previous_cabinet_id',
       u'party_id'],
      dtype='object')

In [70]:
#x=NationalElections.groupby('country_name_short').describe().swaplevel() #.ix['std'].sum()
#x[x==0]
NationalElections[['country_id','country_name_short','country_name','election_id','election_date','seats_total']].ix[0].to_dict()

{'country_id': 33,
 'country_name': 'Australia',
 'country_name_short': 'AUS',
 'election_date': Timestamp('2013-09-07 00:00:00'),
 'election_id': 804,
 'seats_total': 150}

In [90]:
import json

def getPartyJson(code):
    topLevelFields=['country_id','country_name_short','country_name','election_id','election_date','seats_total']
    
    nat = NationalElections[code==NationalElections.country_name_short] \
        .sort_values(['seats','vote_share','party_name_short'],ascending=False)
    r=nat[topLevelFields].reset_index(drop=True).ix[0].to_dict()
    r['election_date']=str(r['election_date'])
    r.update({'parties' : [x[1][nat.columns.difference(topLevelFields)].to_dict() for x in nat.iterrows()]})

    return r


data={code:getPartyJson(code) for code in NationalElections['country_name_short'].drop_duplicates() if not code in ['AUS','CAN','ISR','NZL']}

with open('../public/parlgov.json', 'w') as outfile:
    json.dump(data, outfile,indent=4)

In [88]:
NationalElections.left_right.describe()

count    342.000000
mean       5.125504
std        2.447814
min        0.408900
25%             NaN
50%             NaN
75%             NaN
max        9.824600
Name: left_right, dtype: float64

In [24]:
EuroElections.groupby('country_name_short').apply(getLastElection).reset_index(drop=True)

,country_name_short,country_name,election_type,election_date,vote_share,seats,seats_total,party_name_short,party_name,party_name_english,left_right,country_id,election_id,previous_parliament_election_id,previous_cabinet_id,party_id
0,AUT,Austria,ep,2014-05-25,26.980,5.0,18,OVP,Österreichische Volkspartei,Austrian People's Party,6.4733,59,818,802.0,1070.0,1013
1,AUT,Austria,ep,2014-05-25,24.090,5.0,18,SPO,Sozialdemokratische Partei Österreichs,Social Democratic Party of Austria,3.7293,59,818,802.0,1070.0,973
2,AUT,Austria,ep,2014-05-25,19.720,4.0,18,FPO,Freiheitliche Partei Österreichs,Freedom Party of Austria,8.3494,59,818,802.0,1070.0,50
3,AUT,Austria,ep,2014-05-25,14.520,3.0,18,Gruene,Die Grünen – Die Grüne Alternative,The Greens -- The Green Alternative,2.4604,59,818,802.0,1070.0,1429
4,AUT,Austria,ep,2014-05-25,8.140,1.0,18,NEOS,NEOS – Das Neue Österreich,NEOS -- The New Austria,6.0000,59,818,802.0,1070.0,2255
5,AUT,Austria,ep,2014-05-25,1.180,0.0,18,REKOS,Die Reformkonservativen – REKOS,The Reform Conservatives,7.4000,59,818,802.0,1070.0,2339
6,AUT,Austria,ep,2014-05-25,2.140,0.0,18,KPO,Kommunistische Partei Österreichs,Communist Party of Austria,0.5000,59,818,802.0,1070.0,769
7,AUT,Austria,ep,2014-05-25,2.760,0.0,18,EU-STOP,"EU-Austritt, Direkte Demokratie, Neutralität","EU Withdrawal, Direct Democracy, Neutrality",NaN,59,818,802.0,1070.0,2340
8,BEL,Belgium,ep,2014-05-25,16.790,4.0,21,N-VA,Nieuw-Vlaamse Alliantie,New Flemish Alliance,6.5321,64,832,486.0,987.0,501
9,BEL,Belgium,ep,2014-05-25,12.840,3.0,21,PVV|VLD,Partij voor Vrijheid en Vooruitgang | Vlaamse ...,Party of Liberty and Progress | Flemish Libera...,7.0053,64,832,486.0,987.0,1110
